Importar librerias

In [295]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score
from sklearn.preprocessing import StandardScaler

Cargar los datos

In [296]:
# Cargar el conjunto de datos
data = pd.read_csv('../datasets/water_potability.csv')
data

,ph,Hardness,Solids,Chloramines,Sulfate,Conductivity,Organic_carbon,Trihalomethanes,Turbidity,Potability
0,NaN,204.890455,20791.318981,7.300212,368.516441,564.308654,10.379783,86.990970,2.963135,0
1,3.716080,129.422921,18630.057858,6.635246,NaN,592.885359,15.180013,56.329076,4.500656,0
2,8.099124,224.236259,19909.541732,9.275884,NaN,418.606213,16.868637,66.420093,3.055934,0
3,8.316766,214.373394,22018.417441,8.059332,356.886136,363.266516,18.436524,100.341674,4.628771,0
4,9.092223,181.101509,17978.986339,6.546600,310.135738,398.410813,11.558279,31.997993,4.075075,0
...,...,...,...,...,...,...,...,...,...,...
3271,4.668102,193.681735,47580.991603,7.166639,359.948574,526.424171,13.894419,66.687695,4.435821,1
3272,7.808856,193.553212,17329.802160,8.061362,NaN,392.449580,19.903225,NaN,2.798243,1
3273,9.419510,175.762646,33155.578218,7.350233,NaN,432.044783,11.039070,69.845400,3.298875,1
3274,5.126763,230.603758,11983.869376,6.303357,NaN,402.883113,11.168946,77.488213,4.708658,1


Preparación de los datos

In [297]:
# Seleccionar los features relevantes
#selected_features = ['ph', 'Solids', 'Chloramines', 'Organic_carbon', 'Turbidity', 'Trihalomethanes', 'Potability']
selected_features = ['ph','Chloramines', 'Organic_carbon', 'Turbidity','Trihalomethanes','Potability']
data = data[selected_features].copy()
# Formateo de ph
data[data.isnull().any(axis=1)]
# Se agrega 0 si Potability es 0 y 6 si Potability es 1 (lo aceptable)
data.loc[(data['ph'].isnull()) & (data['Potability'] == 0), 'ph'] = 0
data.loc[(data['ph'].isnull()) & (data['Potability'] == 1), 'ph'] = 6
# Formateo de Trihalomethanes
# Se agrega 100 si es Potability es 0 y 80 si Potability es 1 (lo aceptable)
data.loc[(data['Trihalomethanes'].isnull()) & (data['Potability'] == 0), 'Trihalomethanes'] = 100
data.loc[(data['Trihalomethanes'].isnull()) & (data['Potability'] == 1), 'Trihalomethanes'] = 80
data.shape


(3276, 6)

In [298]:
# Preparing the data

# Remove missing values
#data.dropna(inplace=True)
#selected_features = ['ph', 'Solids','Chloramines', 'Potability']

data = data.append(data[:int(len(data)/0.25)])
data.shape

/var/folders/dm/bzfhlzk90374nx5vf5sn63lh0000gn/T/ipykernel_36263/3184709853.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append(data[:int(len(data)/0.25)])


(6552, 6)

In [299]:
# Dividir los datos en características (X) y etiquetas (y)
X = data.drop('Potability', axis=1)
y = data['Potability']

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [300]:
X

,ph,Chloramines,Organic_carbon,Turbidity,Trihalomethanes
0,0.000000,7.300212,10.379783,2.963135,86.990970
1,3.716080,6.635246,15.180013,4.500656,56.329076
2,8.099124,9.275884,16.868637,3.055934,66.420093
3,8.316766,8.059332,18.436524,4.628771,100.341674
4,9.092223,6.546600,11.558279,4.075075,31.997993
...,...,...,...,...,...
3271,4.668102,7.166639,13.894419,4.435821,66.687695
3272,7.808856,8.061362,19.903225,2.798243,80.000000
3273,9.419510,7.350233,11.039070,3.298875,69.845400
3274,5.126763,6.303357,11.168946,4.708658,77.488213


Creación de los modelos

1- KNN

In [301]:
# Crear y entrenar el modelo de KNN
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train, y_train)

# Predecir las etiquetas de los datos de prueba utilizando el modelo de KNN
knn_pred = knn.predict(X_test)

# Calcular la precisión del modelo de KNN
knn_accuracy = accuracy_score(y_test, knn_pred)
print("Precisión del modelo de KNN:", knn_accuracy)

# Calcular el F1-score del modelo de KNN
knn_f1score = f1_score(y_test, knn_pred)
print("F1-score del modelo de KNN:", knn_f1score)

Precisión del modelo de KNN: 0.6842105263157895
F1-score del modelo de KNN: 0.5909090909090909


2- Decision Tree

In [302]:
# Crear y entrenar el modelo de árboles de decisión
dt = DecisionTreeClassifier(random_state=42)
dt.fit(X_train, y_train)

# Predecir las etiquetas de los datos de prueba utilizando el modelo de árboles de decisión
dt_pred = dt.predict(X_test)

# Calcular la precisión del modelo de árboles de decisión
dt_accuracy = accuracy_score(y_test, dt_pred)
print("Precisión del modelo de árboles de decisión:", dt_accuracy)

Precisión del modelo de árboles de decisión: 0.9115179252479023


3- Logistic Regression

In [303]:
# Crear y entrenar el modelo de regresión logística
lr = LogisticRegression(random_state=42)
lr.fit(X_train, y_train)

# Predecir las etiquetas de los datos de prueba utilizando el modelo de regresión logística
lr_pred = lr.predict(X_test)

# Calcular la precisión del modelo de regresión logística
lr_accuracy = accuracy_score(y_test, lr_pred)
print("Precisión del modelo de regresión logística:", lr_accuracy)

Precisión del modelo de regresión logística: 0.5888634630053394
